### Cross validation Notebook

- [ ] Cora
- [ ] Dennis
- [ ] Manju
- [ ] Corinna
- [ ] GT
- [ ] Cross Coder?

### Colab options

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    !git clone https://github.com/matjesg/DeepFLaSH2.git /content/drive/My\ Drive/DeepFLaSH2
    %cd /content/drive/My\ Drive/DeepFLaSH2
except:
    pass

Import packages

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from skimage import io
import os
from deepflash import unet, preproc, metrics, lr_finder, utils
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import skimage
from skimage.measure import label
from skimage.color import label2rgb
from tqdm import tqdm
%matplotlib inline

Using TensorFlow backend.


## Global Settings

In [8]:
SKIP = 0
CODER = ['gt']  #'rohini', 'dennis', 'cora', 'manju', 'corinna'
NAME_PREFIX = 'falk'
MASK = 'Parv' #cFOS
IMAGE = 'green' #red
CHANNELS_IMG = 1
DATA_PATH = "01_data"
MASK_PATH = "data/labels"
ASSIGNMENT_PATH = 'samples_36_final.csv'
TILE_SHAPE = (540,540)
PADDING = (184,184)
SEED = 0
EL_SIZE = [635.9, 635.9] #micrometers
CHECKPOINTS = 'checkpoints_cv'
LOGDIR = 'logs_cv'

### Training params

In [4]:
PRETAINED = None# 'caffe/caffe_weights.h5' #None
BATCH_NORM = False
EPOCHS = 100
CYCLIC_LR = 'triangular'
SNAPSHOT_INTERVAL = 5
N_SPLITS = 4

### Weighting params

In [5]:
LAMBDA = 50 #50
V_BAL = 0.1 #0.1
SIGMA_BAL = 10 #10 
SIGMA_SEP = 6 #6

## Load Data

Excel list with assignments

In [6]:
assignment = pd.read_csv(ASSIGNMENT_PATH, converters={'Nummer': lambda x: str(x).zfill(4)})
assignment['Group_ID'] = assignment.groupby(['Kondition', 'Area']).ngroup()
file_ids = assignment['Nummer'].tolist()

Images

In [9]:
image_list = [io.imread(os.path.join(DATA_PATH, img_name), as_gray=True) for 
              img_name in [s + '_' + IMAGE + '.tif' for s in file_ids]]

image_list = [np.expand_dims(img, axis=2) for img in image_list]
data = [{'rawdata': img, 'element_size_um': EL_SIZE} for img in image_list]

Masks and weights

In [11]:
coder_weights = {}
coder_masks = {}
for coder in CODER:
    mask_list = [io.imread(os.path.join(MASK_PATH, coder, x), as_gray=True).astype('int')
             for x in [s + '_' + MASK + '.png' for s in file_ids]]
    
    pre = preproc.DataPreProcessor(element_size_um=None,
                               border_weight_sigma_px=SIGMA_SEP,
                               foreground_dist_sigma_px=SIGMA_BAL,
                               border_weight_factor=LAMBDA,
                               foreground_background_ratio=V_BAL
                              )
    weight_list = []
    for i in tqdm(range(len(data))):
        sampleData, sampleLabels, sampleWeights, samplePdf = pre.generateSample(data[i], classlabels=mask_list[i])
        weight_list.append(sampleWeights)  
    coder_masks[coder] = mask_list
    coder_weights[coder] = weight_list 

FileNotFoundError: [Errno 2] No such file or directory: '/media/data/DeepFLaSH2/data/labels/gt/0092_Parv.tif'

for i in range(len(data)):
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    axs[0].imshow(data[i]['rawdata'][...,0],cmap='gray', vmin=0, vmax=1)
    axs[0].set_axis_off()
    axs[1].imshow(coder_masks[coder][i],cmap='gray', vmin=0, vmax=1)
    axs[1].set_axis_off()
    axs[2].imshow(coder_weights[coder][i],cmap='gray')#, vmin=0, vmax=35)
    axs[2].set_axis_off()
    plt.tight_layout()
    plt.show()

### Loop over coder and folds

In [ ]:
for coder in CODER:
    mask_list = coder_masks[coder] 
    weight_list = coder_weights[coder]
    name = NAME_PREFIX + '_' + MASK + '_' + str(CYCLIC_LR) + '_' + coder 
    output_path = os.path.join('output', name)
    if not os.path.isdir(output_path):
        os.makedirs(output_path)
    skf = StratifiedKFold(n_splits=N_SPLITS, random_state=SEED)
    fold = 0
    
    for train_index, test_index in skf.split(assignment['Nummer'], assignment['Group_ID']):
        fold += 1
        if fold < SKIP:
            continue
        X_train, X_test = np.array(image_list)[train_index], np.array(image_list)[test_index]
        y_train, y_test = np.array(mask_list)[train_index], np.array(mask_list)[test_index]
        W_train, W_test = np.array(weight_list)[train_index], np.array(weight_list)[test_index]

        data_train = [{'rawdata': img, 'element_size_um': EL_SIZE} for img in X_train]
        data_test = [{'rawdata': img, 'element_size_um': EL_SIZE} for img in X_test]

        ## Generators
        train_generator = preproc.DataAugmentationGenerator(data = data_train, 
                                                        classlabels=y_train,
                                                        instancelabels=None,
                                                        tile_shape = TILE_SHAPE, 
                                                        padding= PADDING,
                                                        batch_size = 4,
                                                        n_classes=2,
                                                        ignore=None,
                                                        weights=W_train,
                                                        element_size_um=None,
                                                        rotation_range_deg=(0, 360),
                                                        flip=False,
                                                        deformation_grid=(150, 150),
                                                        deformation_magnitude=(10, 10),
                                                        value_minimum_range=(0, 0),
                                                        value_maximum_range=(0.0, 1),
                                                        value_slope_range=(1, 1),
                                                        shuffle=True,
                                                        foreground_dist_sigma_px=SIGMA_BAL,
                                                        border_weight_sigma_px=SIGMA_SEP,
                                                        border_weight_factor=LAMBDA,
                                                        foreground_background_ratio=V_BAL
                                                       )
        test_generator = preproc.TileGenerator(data = data_test,
                                           classlabels=y_test,
                                           instancelabels=None,
                                           tile_shape = TILE_SHAPE, 
                                           padding= PADDING,
                                           n_classes=2,
                                           ignore=None,
                                           weights=W_test,
                                           element_size_um=EL_SIZE,                                       
                                           foreground_dist_sigma_px=SIGMA_BAL,
                                           border_weight_sigma_px=SIGMA_SEP,
                                           border_weight_factor=LAMBDA,
                                           foreground_background_ratio=V_BAL)

        ## Model
        utils.reset_keras()
        name_helper = name + '_' + str(fold)

        print(name_helper)
        
        model = unet.Unet2D(snapshot=None, 
                        n_channels=1, 
                        n_classes=2, 
                        n_levels=4,
                        batch_norm = BATCH_NORM,
                        upsample=False,
                        relu_alpha=0.1,
                        n_features=64, name=name_helper)

        if PRETAINED is not None: 
            model.trainModel.load_weights(PRETAINED,reshape=True, by_name=True)

        model.train(train_generator, 
                validation_generator=test_generator, 
                n_epochs=EPOCHS, 
                snapshot_interval= SNAPSHOT_INTERVAL,
                snapshot_dir = CHECKPOINTS,
                snapshot_prefix=name_helper,
                log_dir = LOGDIR,
                cyclic_lr= CYCLIC_LR)

        ## Predict
        pred_model = unet.Unet2D(snapshot= CHECKPOINTS + '/' +name_helper+'.0100.h5',
                        n_channels=1, 
                        n_classes=2, 
                        n_levels=4, 
                        batch_norm =  BATCH_NORM,
                        upsample=False,
                        relu_alpha=0.1,
                        n_features=64,name="U-Net")

        tile_generator = preproc.TileGenerator(data_test, TILE_SHAPE, PADDING)

        predictions = pred_model.predict(test_generator)
        ## Save
        for i in range(len(predictions[1])):
            idx = np.array(file_ids)[test_index][i]
            file_name = idx + '_' + MASK + '.png'
            io.imsave(os.path.join(output_path, file_name), predictions[1][i])